In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

In [2]:
from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataModule, CHBMITDataset

# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

2025-05-07 19:33:29.061 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /root/snn-encoder-test


In [3]:
datamodule = CHBMITDataModule(dataset, batch_size=512, worker=0)

In [4]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from loguru import logger
import optuna
from optuna.integration import PyTorchLightningPruningCallback
import torch

from eeg_snn_encoder.encoders import BurstEncoder
from eeg_snn_encoder.models.classifier import ModelConfig
from eeg_snn_encoder.models.lightning import LitSeizureClassifier, OptimizerConfig

torch.set_float32_matmul_precision("medium")


def objective(trial: optuna.Trial) -> float:
    model_params: ModelConfig = {
        "threshold": trial.suggest_float("threshold", 0.01, 0.5),
        "slope": trial.suggest_float("slope", 1.0, 20.0),
        "beta": trial.suggest_float("beta", 0.1, 0.99),
        "dropout_rate1": trial.suggest_float("dropout_rate1", 0.1, 0.99),
        "dropout_rate2": trial.suggest_float("dropout_rate2", 0.1, 0.99),
    }

    optimizer_params: OptimizerConfig = {
        "lr": trial.suggest_float("lr", 1e-6, 1e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True),
        "scheduler_factor": trial.suggest_float("scheduler_factor", 0.1, 0.99),
        "scheduler_patience": trial.suggest_int("scheduler_patience", 1, 10),
    }

    max_window = trial.suggest_int("max_window", 4, 16)
    n_max = trial.suggest_int("n_max", 1, max_window)
    t_max = trial.suggest_int("t_max", 0, max_window // n_max)
    t_min = trial.suggest_int("t_min", 0, t_max)

    encoder_params = {
        "max_window": max_window,
        "n_max": n_max,
        "t_max": t_max,
        "t_min": t_min,
    }

    spike_encoder = BurstEncoder(**encoder_params)

    lit_model = LitSeizureClassifier(
        model_config=model_params,
        optimizer_config=optimizer_params,
        spike_encoder=spike_encoder,
    )

    trainer = pl.Trainer(
        max_epochs=20,
        accelerator="auto",
        devices="auto",
        strategy="auto",
        enable_model_summary=False,
        enable_checkpointing=False,
        callbacks=[
            PyTorchLightningPruningCallback(trial, monitor="val_mse"),
            EarlyStopping(monitor="val_loss", mode="min", patience=5),
        ],
        logger=False,
    )

    trainer.fit(lit_model, datamodule=datamodule)

    return trainer.callback_metrics["val_mse"]

In [5]:
import os

# Initialize the Optuna study
sampler = optuna.samplers.TPESampler(multivariate=True, seed=47)
pruner = optuna.pruners.HyperbandPruner()
study = optuna.create_study(
    direction="minimize",
    study_name="model-tuning-be-mse",
    storage=os.environ["OPTUNA_CONN_STRING"],
    load_if_exists=True,
    sampler=sampler,
    pruner=pruner,
)

/root/snn-encoder-test/.venv/lib/python3.12/site-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-05-07 19:33:37,471] A new study created in RDB with name: model-tuning-be-mse


In [ ]:
study.optimize(objective, n_trials=50)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/root/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.
/root/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
logger.info(f"Encoder: Burst Encoding,trial, best_param: {study.best_params}")
logger.info(f"Encoder: Burst Encoding,trial, best_score: {study.best_value}")

2025-05-07 18:15:32.951 | INFO     | __main__:<module>:1 - Encoder: Burst Encoding,trial, best_param: {'threshold': 0.0388955885521671, 'slope': 4.137052363960784, 'beta': 0.6590257321151956, 'dropout_rate1': 0.26198982104313695, 'dropout_rate2': 0.36676998865511823, 'lr': 7.65497513229095e-05, 'weight_decay': 5.318911097556317e-06, 'scheduler_factor': 0.2283003234488558, 'scheduler_patience': 10, 'max_window': 6, 'n_max': 4, 't_max': 0, 't_min': 0}
2025-05-07 18:15:33.428 | INFO     | __main__:<module>:2 - Encoder: Burst Encoding,trial, best_score: 0.8124191761016846


: 